In [ ]:
model_id = "google/gemma-3-12b-it"
print(f"# Model ID: {model_id}")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig
import torch

# 양자화 옵션: 4비트
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    # nf4 양자화

    bnb_4bit_compute_dtype="bfloat16", # 계산시에는 기존 자료형 bf16 사용
    bnb_4bit_use_double_quant=True # 이중 양자화
)


# 토크나이저와 모델 로드
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id,

    torch_dtype=torch.bfloat16, # Gemma 3은 bf16 모델입니다!
    device_map={"":0},  # 0번 GPU에 할당

    quantization_config = quantization_config,
    # 양자화 불필요시 제거

    attn_implementation = 'eager'
    # flash attention 설정
    # Gemma Series가 아닌 경우 'eager'를 "flash_attention_2" 로 변경
    # 미지원/사용하지 않는 경우 제거 (T4 등의 구버전 GPU 등)

)

# if hasattr(model, 'language_model'):
#     model = model.language_model
# Multimodal 모델의 경우, Language Model만 선택할 수도 있으나
# 비전 모델의 사이즈가 매우 작기 때문에 큰 차이는 없음

# Train X (eval)
model.eval()
